In [2]:
!pip install kaggle

In [3]:
from google.colab import drive
drive.mount('/content/drive')
BASE_DIR = '/content/drive/MyDrive/PlantBot'
from google.colab import files
files.upload()

Mounted at /content/drive


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sinchana16","key":"bc41366879312165e713aedf935e26e8"}'}

In [4]:
!mkdir -p ~/.kaggle


In [5]:
!cp kaggle* ~/.kaggle/kaggle.json

In [6]:
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d emmarex/plantdisease -p /content

Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease
License(s): unknown
 93% 614M/658M [00:02<00:00, 228MB/s]
100% 658M/658M [00:03<00:00, 223MB/s]


In [8]:
!unzip -q /content/plantdisease.zip -d /content/plantvillage_data


In [9]:
!ls /content/plantvillage_data/plantvillage/PlantVillage | head

Pepper__bell___Bacterial_spot
Pepper__bell___healthy
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Tomato_Bacterial_spot
Tomato_Early_blight
Tomato_healthy
Tomato_Late_blight
Tomato_Leaf_Mold


In [10]:
import os
import shutil
from sklearn.model_selection import train_test_split
from glob import glob

source_dataset = "/content/plantvillage_data/plantvillage/PlantVillage"
target_base = "/content/PlantDocBot/data"

train_dir = os.path.join(target_base, "train")
val_dir = os.path.join(target_base, "val")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Detect class folders
class_folders = [
    d for d in os.listdir(source_dataset)
    if os.path.isdir(os.path.join(source_dataset, d))
]

print("Detected classes:", class_folders)

for cls in class_folders:
    cls_path = os.path.join(source_dataset, cls)

    # Get all images
    images = (
        glob(cls_path + "/*.JPG")
        + glob(cls_path + "/*.jpg")
        + glob(cls_path + "/*.jpeg")
        + glob(cls_path + "/*.png")
    )

    if len(images) == 0:
        print(f"⚠️ No images found in: {cls}")
        continue

    # Split images
    train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

    # Create class folders inside train/val
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

    # Copy files
    for img in train_imgs:
        shutil.copy(img, os.path.join(train_dir, cls))

    for img in val_imgs:
        shutil.copy(img, os.path.join(val_dir, cls))

print("✅ Dataset split completed successfully!")



Detected classes: ['Pepper__bell___Bacterial_spot', 'Tomato_Bacterial_spot', 'Tomato_healthy', 'Tomato_Leaf_Mold', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato_Septoria_leaf_spot', 'Tomato_Early_blight', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato_Late_blight', 'Tomato__Target_Spot', 'Tomato__Tomato_mosaic_virus', 'Potato___Late_blight', 'Pepper__bell___healthy', 'Potato___healthy', 'Potato___Early_blight']
✅ Dataset split completed successfully!


In [11]:
!ls /content/PlantDocBot/data/train | head

Pepper__bell___Bacterial_spot
Pepper__bell___healthy
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Tomato_Bacterial_spot
Tomato_Early_blight
Tomato_healthy
Tomato_Late_blight
Tomato_Leaf_Mold


In [12]:
!ls /content/PlantDocBot/data/val | head

Pepper__bell___Bacterial_spot
Pepper__bell___healthy
Potato___Early_blight
Potato___healthy
Potato___Late_blight
Tomato_Bacterial_spot
Tomato_Early_blight
Tomato_healthy
Tomato_Late_blight
Tomato_Leaf_Mold


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

train_dir = "/content/PlantDocBot/data/train"
val_dir = "/content/PlantDocBot/data/val"

# Image transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor()
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(val_dir, transform=val_transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

num_classes = len(train_dataset.classes)
print("Detected Classes:", train_dataset.classes)


Detected Classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = models.resnet50(weights="IMAGENET1K_V2")

# Replace final FC layer
model.fc = nn.Linear(model.fc.in_features, num_classes)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


Using device: cuda
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 160MB/s]


In [15]:
epochs = 5  # You can increase to 10–15 later for better accuracy

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {total_loss:.4f}")

print("Training complete!")


Epoch [1/5] - Loss: 199.5983
Epoch [2/5] - Loss: 29.8994
Epoch [3/5] - Loss: 20.6098
Epoch [4/5] - Loss: 13.9548
Epoch [5/5] - Loss: 11.2182
Training complete!


In [16]:
import os
os.makedirs("/content/PlantDocBot/models", exist_ok=True)
print("Models folder created!")


Models folder created!


In [17]:
save_path = "/content/PlantDocBot/models/image_model.pth"
torch.save(model.state_dict(), save_path)

print("Model saved at:", save_path)


Model saved at: /content/PlantDocBot/models/image_model.pth


In [18]:
!ls /content/PlantDocBot/data/train/Tomato_Early_blight | head

0012b9d2-2130-4a06-a834-b1f3af34f57e___RS_Erly.B 8389.JPG
0034a551-9512-44e5-ba6c-827f85ecc688___RS_Erly.B 9432.JPG
00c5c908-fc25-4710-a109-db143da23112___RS_Erly.B 7778.JPG
00e7c4b2-3005-4558-9cfa-235e356cb7a8___RS_Erly.B 7844.JPG
0114f7b3-7f56-4bef-aa47-7a82da142d4b___RS_Erly.B 7437.JPG
011aa278-a39e-40b0-b6e4-2e2f835889f0___RS_Erly.B 8251.JPG
01861c93-ea8b-4820-aaa8-cc6003b3e75b___RS_Erly.B 7855.JPG
01f87f04-ff38-4aba-b7d1-587aa06e5282___RS_Erly.B 7851.JPG
0208ed52-1b28-4c9c-88e3-4667581e1109___RS_Erly.B 7829.JPG
023fe2d4-6e1d-40ce-99ae-85ba90f436ff___RS_Erly.B 6354.JPG


In [19]:
import torch
from torchvision import transforms, models
from PIL import Image
import json
import os

# Load class names
train_dir = "/content/PlantDocBot/data/train"
class_names = sorted(os.listdir(train_dir))

def load_image_model():
    num_classes = len(class_names)

    model = models.resnet50(weights="IMAGENET1K_V2")
    model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

    model_path = "/content/PlantDocBot/models/image_model.pth"
    model.load_state_dict(torch.load(model_path, map_location="cpu"))
    model.eval()

    return model

image_model = load_image_model()

# Image transform
img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

def predict_from_image(image_path):
    image = Image.open(image_path).convert("RGB")
    img_tensor = img_transform(image).unsqueeze(0)

    with torch.no_grad():
        outputs = image_model(img_tensor)
        _, predicted = torch.max(outputs, 1)

    return class_names[predicted.item()]


In [20]:
from google.colab import files
uploaded = files.upload()

for filename in uploaded.keys():
    prediction = predict_from_image(filename)
    print("Prediction:", prediction)


Saving 00416648-be6e-4bd4-bc8d-82f43f8a7240___GCREC_Bact.Sp 3110.JPG to 00416648-be6e-4bd4-bc8d-82f43f8a7240___GCREC_Bact.Sp 3110.JPG
Prediction: Tomato_Bacterial_spot


In [21]:
disease_treatments = {
    "Tomato_Bacterial_spot": "Use copper-based fungicides, remove infected leaves, avoid overhead watering.",
    "Tomato_Early_blight": "Apply fungicides containing chlorothalonil. Practice crop rotation.",
    "Tomato_Late_blight": "Use systemic fungicides like metalaxyl. Remove infected plants completely.",
    "Tomato_Leaf_Mold": "Increase air circulation, apply fungicides like chlorothalonil.",
    "Tomato_healthy": "Plant appears healthy. Continue regular care.",

    "Potato___Early_blight": "Use fungicides like mancozeb or chlorothalonil. Improve soil drainage.",
    "Potato___Late_blight": "Apply copper fungicides. Destroy infected leaves immediately.",
    "Potato___healthy": "Plant is healthy.",

    "Pepper__bell___Bacterial_spot": "Use bactericides like copper. Remove infected plant parts.",
    "Pepper__bell___healthy": "Healthy plant.",

    "Apple___Black_rot": "Prune infected branches. Apply fungicides like captan or thiophanate-methyl.",
    "Apple___healthy": "Tree looks healthy."
}


In [22]:
def plantdocbot(image_path=None, text=None):
    result = {}

    # If image provided
    if image_path:
        disease = predict_from_image(image_path)
        treatment = disease_treatments.get(disease, "Treatment information not available.")
        result["type"] = "image"
        result["diagnosis"] = disease
        result["treatment"] = treatment

    # If text provided
    if text:
        prediction = predict_from_text(text)
        treatment = disease_treatments.get(prediction, "Treatment information not available.")
        result["type"] = "text"
        result["diagnosis"] = prediction
        result["treatment"] = treatment

    return result


In [25]:
response = plantdocbot(image_path="/content/plantvillage_data/PlantVillage/Tomato_Bacterial_spot/00416648-be6e-4bd4-bc8d-82f43f8a7240___GCREC_Bact.Sp 3110.JPG")
print(response)


{'type': 'image', 'diagnosis': 'Tomato_Bacterial_spot', 'treatment': 'Use copper-based fungicides, remove infected leaves, avoid overhead watering.'}
